## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


# 1. BUSINESS UNDERSTANDING 
## Business Problem

The entertainment industry is undergoing rapid transformation, with original content emerging as a key driver of audience engagement and revenue growth. Major players like Netflix, Amazon, and other sources are heavily investing in original films —reaping substantial financial returns and strengthening their brand presence.
Recognizing this trend, Flix company has made the strategic decision to launch a new movie studio. However, it currently lacks the data-driven insights necessary to understand what factors contribute to a film’s box office success.
As data scientists, our role is to explore publicly available movie performance data to uncover patterns that indicates
what makes a movie financially successful. The goal is to provide clear data- driven, actionable recommendations that will help guide decisions about genre, budget size, release timing, and other production choices.



## Stakeholders and Use Cases
Primary Stakeholder:
Head of the New Movie Studio

Use Case:
Leverage data-driven insights to inform strategic decisions on film production. This includes identifying high-performing genres, determining optimal budget ranges, selecting ideal release windows, and shaping casting strategies—all aimed at maximizing box office success and return on investment.

## Project Objectives
* Identify which genres perform best at the box office, considering revenue and profitability.

* Analyze the impact of budget, runtime, cast, and release month on a film’s success.

* Provide actionable recommendations for the types of films the company should produce.

## Conclusion: Implications and Recommendations

This project provides a data-driven foundation to support the successful launch of Flix company’s new movie studio while reducing financial risk. By uncovering the key factors that correlate with box office success, the Head of Studio is equipped to make informed, strategic decisions, including:

* Genre Selection: Focus on genres with a strong track record of performance.

* Budget Planning: Allocate production budgets based on historically successful investment ranges.

* Release Strategy: Optimize release timing to align with peak audience engagement periods.

* Talent Strategy: Identify the cast and crew characteristics commonly linked to high-grossing films.



# 2. DATA UNDERSTANDING

## Data Sources

This project uses data from three high-quality, complementary sources of movie data:

### 1. Box Office Mojo (bom.movie_gross.csv.gz)

Provides domestic box office revenue data.

Includes key features such as: title, studio, domestic_gross, release_date, and year.

used to determine the financial performance of films.

 ### 2. IMDb(Internet Movie Database) (im.db.zip)

Contains detailed metadata about films and user-ratings.

Key tables used:

* movie_basics: Includes primary_title, original_title, genres, runtime_minutes, and start_year.

* movie_ratings: Contains user rating data (average_rating, num_votes).

used for movie characteristics and audience quality perceptions.

 ### 3. TheMovieDB (TMDb) (tmdb.movies.csv.gz)

TheMovieDb entails the following:
* User-generated popularity 
* voting data

Key features: title, popularity, vote_average, vote_count, release_date, genres, budget, revenue

Purpose: Complements Box Office Mojo and IMDb with:

Popularity metrics: Show which films gain audience traction pre- and post-release

Vote data: Allows cross-comparison with IMDb ratings


### 4. The Numbers (tn.movie_budgets.csv.gz)

It consits of  Film production budgets and worldwide gross

Key Features:

* Release_date, movie, production_budget, domestic_gross, worldwide_gross

Why It Matters:

* Gives a complete financial picture by providing both the cost of making the film (production budget) and revenue generated globally.

* Allows calculation of Return on Investment (ROI) — one of the most important metrics when deciding which types of films to produce.


# 3. DATA PREPARATION

To determine which types of films perform best at the box office, it is essential to integrate and clean datasets from Box Office Mojo, IMDb, and The Movie Database (TMDb). The following section outlines the data preparation process, including code examples and explanations for each step.

## Loading and inspecting the Raw data 

Befoe we load our datasets , we will import libraries which we will use.


In [34]:
# Importing necessary libraries
import pandas as pd 
import sqlite3
import zipfile


The next step would be to load our datasets and see what it entails .

In [35]:
# loading the dataset of Box Office Mojo
box_office_mojo= pd.read_csv('zippedData/bom.movie_gross.csv/bom.movie_gross.csv')
# displays the first few rows 
box_office_mojo.head()


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


The dataset contains the domestic office box revenue which we will use as the target variable.  
We'll filter for movies only (not re-releases or limited runs), and ensure all rows have valid gross data.

In [37]:
# Loading the sqlite database of IMDB
# Connecting to sqlite IMDB database
# Step 1: Extract the .db file from the zip archive
# with zipfile.ZipFile('zippedData/im.db.zip', 'r') as zip_ref:
#     zip_ref.extractall('zippedData')  # This will extract im.db into zippedData/

# Step 2: Connect to the extracted SQLite database
conn = sqlite3.connect('zippedData/im.db')

# Step 3: Load tables into pandas DataFrames
basics = pd.read_sql('SELECT * FROM movie_basics', conn)
ratings = pd.read_sql('SELECT * FROM movie_ratings', conn)
 # Close connection
conn.close()

The IMDB database consists of :
movie_basics: includes metadata like genres, runtime_minutes, and start_year.

movie_ratings: includes average_rating and num_votes.

These are used to describe each film and estimate perceived quality and popularity.

In [40]:
# Loading TheMovieDB
movie_Db= pd.read_csv('zippedData/tmdb.movies.csv/tmdb.movies.csv',index_col=0 )
# Views the first few rows
movie_Db.head()
 #movie_Db.info()
# movie_Db.shape

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


TheMovieDb adds user interest indicators (popularity, vote_average, vote_count) and financial info (budget, revenue) not found in the other sources.

In [42]:
# Load movie budget dataset (TSV format, gzip-compressed)
movie_budget = pd.read_csv('zippedData/tn.movie_budgets.csv/tn.movie_budgets.csv') #sep='\t')

# Display the first few rows
movie_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
